In [ ]:
import polars as pl

%load_ext autoreload
%autoreload 2

# Base Load Analysis Demo with Polars

This notebook demonstrates analyzing base load (standby power consumption) using the BaseloadAnalyzer class. Base load represents the minimum continuous power draw in a system.

## Key Metrics
1. Base load value in WATTS - Consistent minimum power draw
2. Energy consumption in kWh - Power used over time
3. Base load percentage - Portion of total consumption that is baseline

## Data Requirements
Input data (LazyFrame):
- timestamp: datetime with timezone
- total: energy readings in kWh (15-minute intervals)


> ⚠️**Note:** we have several example files available,  
> *energy_use_big* is from a giant building with incredible base load.  
> *energy_use_test1* is from a regular family residence.


In [14]:
from openenergyid.baseload.analysis import BaseloadAnalyzer

# Define schema for data loading
schema = {"timestamp": pl.Datetime(time_zone="Europe/Brussels"), "total": pl.Float64}

# Load example data with schema
energy_data = pl.scan_ndjson("data/PP/energy_use_test1.ndjson", schema=schema)

## Initialize Analyzer
Set up analyzer with timezone and quantile settings

In [15]:
# Create analyzer (5% quantile = ~72 min of lowest daily values)
analyzer = BaseloadAnalyzer(timezone="Europe/Brussels", quantile=0.15)

# Convert energy readings to power series
power_data = analyzer.prepare_power_seriespolars(energy_data)

## Analyze at Different Time Scales
Demonstrate flexibility in analysis periods

In [ ]:
# Analyze at different granularities
hourly = analyzer.analyze(power_data, "1h").collect()
daily = analyzer.analyze(power_data, "1d").collect()
monthly = analyzer.analyze(power_data, "1mo").collect()

# Show monthly summary
print("Monthly Base Load Analysis:")
print(monthly.select(["timestamp", "average_daily_baseload_in_watt", "baseload_ratio"]).head())

## Visualization Example
Plot daily base load vs total consumption

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

# Convert to pandas for plotting
daily_pd = daily.to_pandas()

# Create figure with basic lines
fig = px.line(
    daily_pd,
    x="timestamp",
    y=["consumption_due_to_baseload_in_kilowatthour", "total_consumption_in_kilowatthour"],
    title="Daily Base Load vs Total Consumption",
    labels={"value": "Energy (kWh)", "variable": "Type"},
)

# Add average lines
fig.add_hline(
    y=daily_pd["consumption_due_to_baseload_in_kilowatthour"].mean(),
    line_dash="dash",
    line_color="blue",
    annotation_text="Average Base Load",
)

fig.add_hline(
    y=daily_pd["total_consumption_in_kilowatthour"].mean(),
    line_dash="dash",
    line_color="red",
    annotation_text="Average Total Consumption",
)

# Update colors and legend
fig.update_traces(
    name="Base Load",
    line_color="blue",
    selector=dict(name="consumption_due_to_baseload_in_kilowatthour"),
)
fig.update_traces(
    name="Total Consumption",
    line_color="red",
    selector=dict(name="total_consumption_in_kilowatthour"),
)

fig.show()

In [ ]:
import polars as pl
import plotly.express as px

# Load data with schema
schema = {"timestamp": pl.Datetime(time_zone="Europe/Brussels"), "total": pl.Float64}
energy_data = pl.scan_ndjson("data/PP/energy_use_test1.ndjson", schema=schema).sort("timestamp")


# Analyze with different quantiles
def analyze_quantile(q: float):
    analyzer = BaseloadAnalyzer(timezone="Europe/Brussels", quantile=q)
    power_data = analyzer.prepare_power_seriespolars(energy_data)
    return analyzer.analyze(power_data, "1d").collect()


# Get results for different quantiles
q05 = analyze_quantile(0.05)
q10 = analyze_quantile(0.10)
q15 = analyze_quantile(0.15)

# Create visualization
fig = go.Figure()

# Add lines for each quantile
for data, q in [(q05, "5%"), (q10, "10%"), (q15, "15%")]:
    fig.add_trace(
        go.Scatter(
            x=data["timestamp"],
            y=data["consumption_due_to_baseload_in_kilowatthour"],
            name=f"Base Load (q={q})",
            mode="lines",
        )
    )

# Add total consumption line
fig.add_trace(
    go.Scatter(
        x=q05["timestamp"],
        y=q05["total_consumption_in_kilowatthour"],
        name="Total Consumption",
        mode="lines",
        line=dict(color="gray", dash="dot"),  # Make it dotted gray to distinguish
    )
)

fig.update_layout(
    title="Base Load Comparison - Different Quantiles",
    xaxis_title="Date",
    yaxis_title="Energy (kWh)",
)

fig.show()

# Print average values
print("\nAverage Base Load (kWh):")
print(f"5% quantile: {q05['consumption_due_to_baseload_in_kilowatthour'].mean():.3f}")
print(f"10% quantile: {q10['consumption_due_to_baseload_in_kilowatthour'].mean():.3f}")
print(f"15% quantile: {q15['consumption_due_to_baseload_in_kilowatthour'].mean():.3f}")

In [ ]:
monthly
# daily

## Key Insights
- Base load typically accounts for 20-40% of total consumption
- Higher ratios may indicate energy saving opportunities
- Analysis maintains timezone awareness throughout